In [1]:
# Before run this codes, increase data rate by jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
pd.options.display.max_rows = 150
pd.options.mode.chained_assignment = None  # default='warn'

#create the list of dictinary formatted strings by extracting data item
#from twitter API scrape
#files = ['twtgun3.24.1.txt', 'twtgun3.24.2.txt','twtgun3.24.3.txt', 'twtgun3.24.4.txt']

group = []
for i in range(1,11):
   
    with open('twtgun3.24.' + str(i) + '.txt') as fhand:
        for item in fhand:
            dic = {}
            x = re.findall("^{'created_at': \'(.*?)\'", item)
            if len(x) > 0:
                dic["created_at"] = str(x[0])
            else:
                dic["created_at"] = None
            y = re.findall("{'full_text': \'(.*?)\'", item)
            if len(y) > 0:
                dic["full_text"] = str(y[0])
            else:
                dic["full_text"] = None
            z = re.findall(" 'text': \'(.*?)\'", item)
            if len(z) > 0:
                dic["text"] = str(z[0])
            else:
                dic["text"] = None
            a = re.findall("\'location\': \'(.*?)\'", item)
            if len(a) > 0:
                dic["location"] = str(a[0])
            else:
                dic["location"] = None
            b = re.findall("\'time_zone\': \'(.*?)\'", item)
            if len(b) > 0:
                dic["time_zone"] = str(b[0])
            else:
                dic["time_zone"] = None
            group.append(dic)

    fhand.close()

#convert the list into json object

with open('data.json', 'w') as fp:
    json.dump(group, fp)

# convert json file into pandas DataFrame
df = pd.read_json('data.json')

FileNotFoundError: [Errno 2] No such file or directory: 'twtgun3.24.1.txt'

In [ ]:
df.sample(5)  # To check the data
#df.to_csv('df.csv')

In [ ]:
df2 = df.reindex(columns = ['created_at','location','time_zone','text','full_text'])
pd.options.display.max_rows=10

In [ ]:
df2['text'] = df2['text'].replace(r'\\n', '', regex=True).str.lower()
df2['text'] = df2['text'].str.strip()
df2['full_text'] = df2['full_text'].replace(r'\\n', '', regex=True).str.lower()
df2['full_text'] = df2['full_text'].str.strip()

In [ ]:
print(df2.head())    # To check the data

In [ ]:
#df2['full_text'].str.findall('\\n') just to make sure no data will be found

In [ ]:
#make copy of df2 and name it df3
df3 = df2

In [ ]:
#Take 'full_text' when available, otherwise take 'text'
df3['tweet'] = df3['full_text'].where(pd.notna(df3['full_text']), other = df3['text'])

In [ ]:
pd.isna(df3['tweet']).sum()  # Check how many 'tweet' observations are missing

In [ ]:
#drop columns 'text' & 'full_text'
df3 = df3.drop(['text', 'full_text'], axis=1)
#Extract hashtag from 'tweet' and create a new cloumn 'hashtag'
df3['hashtag'] = df3['tweet'].str.findall(r'#.*?\s')  # .findall returns list which causes issues later
df3.reindex(columns = ['created_at','location','time_zone','tweet', 'hashtag'])

In [ ]:
#drop rows where all columns are NaN
df3 = df3.dropna(how='all')
print(df3.sample(5))

In [ ]:
# Convert'created_at' time data rounding to nearest minute 
df3['created_at'] = df3['created_at'].apply(lambda x: x.round('min'))

In [ ]:
# Check how many tweets creatd every minute during the data collection period
# At this point I just want to check the time trend of the tweets which can be done without time-zone conversion.
df3.groupby('created_at').count()['tweet']

In [ ]:
#UTC is 5 hours earlier than EST. So in the graph below, tweets were peaked around 15:30pm
df3.groupby('created_at').count()['tweet'].plot()

In [ ]:
# As the next step, use regular expression to tokenize tweets
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

In [ ]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [ ]:
# Test run using only one tweet
tweet = df3['tweet'][10]
print(preprocess(tweet))   #success!

In [ ]:
# Create a new dataframe by dropping rows with NA data
df4 = df3.dropna(axis=0, inplace=False)
df4.head()

In [ ]:
# Tokenize tweet data in the dataframe
df4['preprocess_tweet'] = df4['tweet'].apply(lambda x : preprocess(x))
df4.head()

In [ ]:
#from nltk download premade stopwords list
from nltk.corpus import stopwords
import string

punc = list(string.punctuation)
stop = stopwords.words('english') + punc + ['rt', 'via', '’', 'amp']

In [ ]:
#Test run using only one tweet
terms_nostop = [term for term in preprocess(tweet) if term not in stop]  #success!

In [ ]:
# There are 37487 rows in df4. for each item (list) in the 'preprocess_tweet' column, check if it's non-empty, then 
# remove stop words from the item and append it to the longer 'items' list. Since the data is large, 
# increase data rate by jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

items = []
for item in df4['preprocess_tweet']:
    if len(item) !=0:
        for i in item:
            if i not in stop and not i.startswith(('#', '@')):
                items.append(i.strip())
            else:
                continue  
    else:
        continue
        
print(items[:10])

In [ ]:
# Listing top 30 most common words in tweets during the data collection period
import operator
from collections import Counter

count_all = Counter()
count_all.update(items)
print(count_all.most_common(30))    

In [ ]:
# Creating the list of hashtags tweeted during the data colletion period
hashlist = []
for item in df4['hashtag']:
    if len(item) !=0:
        for i in item:
            if i.startswith('#'):
                hashlist.append(i.strip())
            else:
                continue  
    else:
        continue
        
print(hashlist[:10])

In [ ]:
#control_pattern = re.compile(r'#guncontrol[\.]|#guncont[r]?ol[\w!]+|#gunreform[.\w]*')
control_pattern = re.compile(r'#guncont[r]?ol[\w!]+|#gunreform[.\w]*')                                
clean_hashlist =[]
for item in hashlist:
    i = control_pattern.sub(r'#guncontrol', item)
    clean_hashlist.append(i)
    
march_pattern = re.compile(r'#march4ourlives|#marchforourlives!|#marchforourlives[\w.]+')

clean_hashlist2 =[]
for item in clean_hashlist:
    i = march_pattern.sub(r'#marchforourlives', item)
    clean_hashlist2.append(i)
    
enough_pattern = re.compile(r'#enough[.\w]+')

clean_hashlist3 =[]
for item in clean_hashlist2:
    i = enough_pattern.sub(r'#enough', item)
    clean_hashlist3.append(i)

In [ ]:
# Listing top 30 most popular hash tags during the data collection period

count_hash = Counter()
count_hash.update(clean_hashlist3)
print(count_hash.most_common(30))

# "#guncontrol', '#guncontrol.', and '#guncontrolnow' should be count toghether. So should '#marchforourlives' and '#march4ourlives' etc

In [ ]:
# to remove emojis from tokens, compile emoji patterns to be removed- Unicode some emoji code -shorter ones- cannot
# be compiled. Why?

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F500"  # symbols & pictographs
        u"\U0001F520-\U0001F52F"
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)                          
        u"\U0001F910-\U0001F96B" 
        u"\U0001F52B"
        u"\U0001F5E3"             # speech
        u"\U0001F5F3"             # vote 
        u"\U0001F91B-\U0001F939"                   
        u"\U0001F191-\U0001F19A"
        u"\U0001F595"
                             "]+", flags = re.UNICODE)

In [ ]:
# Remove emojis from the list of tokenized words
items_noemoji = []
for item in items:
    i = emoji_pattern.sub(r'', item)
    items_noemoji.append(i)
    
print(items_noemoji[:10])

In [ ]:
# Listing top 30 word-pairs tweeted together during the data collection periods
from nltk import bigrams

terms_bigram = list(bigrams(items_noemoji))
count_bigram = Counter()
count_bigram.update(terms_bigram)
print(count_bigram.most_common(30))

In [ ]:
#Visualize top 30 popular hashtags

import vincent
vincent.core.initialize_notebook()

hash_freq = count_hash.most_common(30)  #create list of tuples
labels, freq = zip(*hash_freq)  #seperate the above into 1. tuple of labels & 2. tuple of counts
data = {'data': freq, 'x': labels} #create dictionary of tuples
bar = vincent.Bar(data, iter_idx='x')
bar.display()

In [ ]:
# Create the list of time stamps when #hermosabeach was tweeted
beach_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#hermosabeach' in df4['tweet'].iloc[i]:
        beach_hash_time.append(df4['created_at'].iloc[i])
        
print(beach_hash_time[:10])

In [ ]:
# Creating datetimeindex for time series data for pandas. Twitter streaming data is based on UTC time.
# From US & Canada ETS time, it's 4 hours earlier. Adjust for daytime savings.

from pandas.tseries.offsets import Hour
one_hour = Hour(1)

idx = pd.DatetimeIndex(beach_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(beach_hash_time)
#ones
beach_hash = pd.Series(ones, index=idx_est_ds)
per_minute = beach_hash.resample('1min').sum().fillna(0)

In [ ]:
#Visualize the frequency of #hermosabeach was tweeted during the data collection period
time_chart = vincent.Line(per_minute)
time_chart.axis_titles(x='Time', y='Hashtag frequencies')
time_chart.display()

In [ ]:
# Prepar hashtag #marchforourlives for the same analysis
march_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#marchforourlives' in df4['tweet'].iloc[i]:
        march_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(march_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(march_hash_time)
#ones
march_hash = pd.Series(ones, index=idx_est_ds)
march_per_minute = march_hash.resample('1min').sum().fillna(0)

In [ ]:
# Prepar hashtag #enough for the same analysis
enough_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#enough' in df4['tweet'].iloc[i]:
        enough_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(enough_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(enough_hash_time)
#ones
enough_hash = pd.Series(ones, index=idx_est_ds)
enough_per_minute = enough_hash.resample('1min').sum().fillna(0)

In [ ]:
# Prepar hashtag #neveragain for the same analysis
again_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#neveragain' in df4['tweet'].iloc[i]:
        again_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(again_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(again_hash_time)
#ones
again_hash = pd.Series(ones, index=idx_est_ds)
again_per_minute = again_hash.resample('1min').sum().fillna(0)

In [ ]:
# Prepar hashtag #guncontrol for the same analysis
control_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#guncontrol' in df4['tweet'].iloc[i]:
        control_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(control_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(control_hash_time)
#ones
control_hash = pd.Series(ones, index=idx_est_ds)
control_per_minute = control_hash.resample('1min').sum().fillna(0)

In [ ]:
# Prepar hashtag #gunviolence for the same analysis
violence_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#gunviolence' in df4['tweet'].iloc[i]:
        violence_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(violence_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(violence_hash_time)

violence_hash = pd.Series(ones, index=idx_est_ds)
violence_per_minute = violence_hash.resample('1min').sum().fillna(0)

In [ ]:
# Prepar hashtag #schoolshooting for the same analysis
school_hash_time =[]

for i in range(len(df4['tweet'])):
    if '#schoolshooting' in df4['tweet'].iloc[i]:
        school_hash_time.append(df4['created_at'].iloc[i])
        
idx = pd.DatetimeIndex(school_hash_time)
idx_local = idx.tz_localize(tz='UTC')
idx_est = idx_local.tz_convert(tz='US/Eastern')
idx_est_ds = idx_est - one_hour    #daytime savings adjustment
ones = [1]*len(school_hash_time)
#ones
school_hash = pd.Series(ones, index=idx_est_ds)
school_per_minute = school_hash.resample('1min').sum().fillna(0)

In [ ]:
#Visualize the frequency of #shoolshooting was tweeted
time_chart = vincent.Line(school_per_minute)
time_chart.axis_titles(x='Time', y='Hashtag frequencies')
time_chart.display()

In [ ]:
match_data = dict(Beach=per_minute, March=march_per_minute, Enough=enough_per_minute, Again=again_per_minute, Control=control_per_minute, Violence=violence_per_minute)
all_matches = pd.DataFrame(data = match_data,index=march_per_minute.index)
all_matches

In [ ]:
#Visualize and comparing the frequencies of the top 3 the most popular hashtags during the data collection time
time_chart_top3 = vincent.Line(all_matches[['March', 'Enough', 'Control']])
time_chart_top3.axis_titles(x='Time', y='Freq')
time_chart_top3.legend(title='Top 3 Most popular hashtags')
time_chart_top3.display()

In [ ]:
#Visualize and comparing the frequencies of the next 3 of the most popular hashtags during the data collection time
time_chart_next3 = vincent.Line(all_matches[['Violence', 'Again', 'Beach']])
time_chart_next3.axis_titles(x='Time', y='Freq')
time_chart_next3.legend(title='Top 4 to 6 Most popular hashtags')
time_chart_next3.display()

In [ ]:
# Analyze number of tweets by geographic locations. Within USA, broken down by states
df5 = df4[(df4['time_zone']=='Pacific Time (US & Canada)')|(df4['time_zone']=='Eastern Time (US & Canada)')|(df4['time_zone']=='Central Time (US & Canada)')|(df4['time_zone']=='Mountain Time (US & Canada)')]

In [ ]:
df5['processed_location'] = df5['location'].apply(lambda x: emoji_pattern.sub(r'', x))
hashtag = re.compile(r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)')
number = re.compile(r'(?:(?:\d+,?)+(?:\.?\d+)?)')

df5['processed_location2'] = df5['processed_location'].apply(lambda x: hashtag.sub(r'', x))
df5['processed_location3'] = df5['processed_location2'].apply(lambda x: number.sub(r'', x))
df5 = df5.drop(['processed_location2', 'processed_location'], axis=1)

In [ ]:
def pattern(text):
    x = re.compile(r'[\w\s]+[\s]?[\w]+, ([A-Z][A-Z]+)')
    match = x.search(text)   #return match object
    if match != None:
        return match.group(1)
    else:
        pass

In [ ]:
df5['clean_states'] = df5['processed_location3'].apply(lambda text: pattern(text))

In [ ]:
def pattern2(text):
    x = re.compile(r'([\w\s]+[\s]?[\w]+), US[A]?|[\w]+,(Texas), EE.UU.|[\w\s]+[\w]+,(Florida)|(NH)\s USA|Cape\s Cod,(Ma)|Hastings,\s(NE)B|(NY),way|Athens,\s(Georgia)')
    match = x.search(text)
    if match != None:
        return match.group(1)
    else:
        pass

In [ ]:
df5['clean_states2'] = df5['processed_location3'].apply(lambda text : pattern2(text))

In [ ]:
#df5['clean_states'].to_csv('name2.csv')
#df5['clean_states2'].to_csv('name4.csv')

In [ ]:
d = {'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ','Arkansas':'AR', 'California':'CA', 'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL', 'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY', 'Louisiana':'LA', 'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN', 'Mississippi':'MS', 'Missouri':'MO', 'Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV', 'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY', 'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK', 'Oregon':'OR', 'Pennsylvania':'PA', 'Rhode Island':'RI', 'South Carolina':'SC', 'Tennessee':'TN', 'Texas':'TX', 'Utah':'UT', 'Vermont':'VT', 'Virginia':'VA', 'Washington':'WA', 'West Virginia':'WV', 'Wisconsin':'WI', 'Wyoming':'WY', 'American Samoa':'AS', 'District of Columbia':'DC', 'Federated States of Micronesia':'FM', 'Guam':'GU', 'Marshall Islands':'MH', 'Northern Mariana Islands':'MP', 'Palau':'PW', 'Puerto Rico':'PR', 'Virgin Islands':'VI'}
df5['clean_states2'] = df5['clean_states2'].str.strip()
df5['clean_states2'] = df5['clean_states2'].replace(d)     

In [ ]:
df5['clean_states2']=df5['clean_states2'].replace(to_replace=[r'[\w]+\sNew\sYork', r'DC\s[\w\s]+', r'[\w]+\sKentucky', r'[\w]+\sWisconsin', r'[\w]+\sTexas', r'[\w]+\sCalifornia', r'[\w]+\sFlorida',r'[\w]+\sSC',r'[\w]+\sMichigan',r'[\w]+\sIndiana',r'[\w\s]+\sNew\sJersey',r'[\w]+\sWashington\sState',r'Virginia\sand\sFlorida',r'[\w\s]*Jersey\sShore'], value=['NY','DC','KY','WI','TX','CA','FL','SC','MI','IN','NJ','WA','FL','NJ'], regex=True)
df5['clean_states2']=df5['clean_states2'].replace(to_replace =['TEXAS','Northern Virginia','Nueva York','ARIZONA','Tenn','South Dakota'], value=['TX','VA','NY','AZ','TN','SD'])
df5['clean_states2']=df5['clean_states2'].replace(to_replace =['Boston','Albuquerque','Dallas','Seattle','Philly','Brooklyn','Phoenix'], value=['MA','NM','TX','WA','PA','NY','AZ'])
df5['clean_states2']=df5['clean_states2'].replace(to_replace =['West Coast','Southeast','New England','Midwest','Left Coast','Canada'], value=['','','','','',''])

In [ ]:
df5['states'] = np.where(df5['clean_states'] !="USA", df5['clean_states'], df5['clean_states2'])
df5['states']=df5['states'].replace(to_replace =['CALIFORNIA','EE','GODS','NOT','NEB','OHIO','TEXAS'], value=['CA','TX','NH','NY','NE','OH','TX'])

In [ ]:
df5.to_csv('text.csv')